In [16]:
import pandas as pd
from time import time
import numpy as np

In [17]:
start = time()
pkls =pd.read_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet_inroi.ftr') #42sec local # 90sec network
print(round(time()-start),'seconds')

82 seconds


In [18]:
# pkls=pkls[pkls['inroi']>0].reset_index(drop=True)
# pkls.to_feather(r'\\fatherserverdw\q\research\images\skin_aging\wsi\hovernet_out\df\2d_skin_hovernet_inroi.ftr')

In [19]:
pkls['inroi'].unique(),pkls['type'].unique()

(array([1, 3, 4, 2, 5, 6], dtype=int64),
 array([10,  9,  8,  2,  7,  5,  4,  6, 11,  3], dtype=int64))

In [20]:
agelut = pd.read_excel(r'\\fatherserverdw\Q\research\source code\skin_aging_matlab\skin_samples.xlsx', sheet_name='imlist')

In [21]:
len(pkls)

7856729

In [22]:
pkls.head()

,bbox,centroid,contour,type,inroi,resident_area,c2t_distance,Area,Perimeter,Circularity,...,orientation,AspectRatio,Sol,Extent,EquiDia,imID,dist2nearest,oriA,oriB,local_align
0,"[[39028, 23927], [39050, 23938]]","[40315.561643835616, 22653.72602739726]","[[40316, 22644], [40316, 22646], [40315, 22647...",10,1,269182,"[15, 223, 129, 163, 214, 155, 119, 91, 177, 0,...",121.5,54.041630,0.522792,...,18.977255,0.362021,0.886861,0.502066,12.437789,8,33.837849,27.686026,39.083927,0.352779
1,"[[39049, 23898], [39066, 23912]]","[40287.50326797386, 22672.287581699347]","[[40285, 22665], [40285, 22666], [40283, 22668...",10,1,269182,"[6, 212, 132, 166, 218, 171, 132, 94, 191, 0, 27]",131.5,48.041630,0.715979,...,27.686026,0.653820,0.929329,0.552521,12.939513,8,33.837849,18.977255,39.083927,0.352779
2,"[[39052, 24088], [39066, 24102]]","[40479.20183486238, 22674.963302752294]","[[40481, 22668], [40477, 22672], [40477, 22673...",10,1,269182,"[90, 39, 190, 203, 5, 88, 133, 131, 156, 0, 200]",90.5,42.041630,0.643427,...,46.967815,0.432951,0.932990,0.461735,10.734439,8,52.000000,60.451813,84.023666,0.293905
3,"[[39069, 23255], [39082, 23267]]","[39644.69158878505, 22691.168224299065]","[[39645, 22685], [39643, 22687], [39643, 22688...",10,1,269182,"[243, 192, 213, 246, 153, 197, 3, 59, 164, 0, 53]",89.5,39.213203,0.731422,...,36.164223,0.603901,0.937173,0.573718,10.674968,8,72.622311,58.224380,104.293274,0.524896
4,"[[39076, 23171], [39086, 23187]]","[39562.37096774193, 22696.935483870966]","[[39559, 22692], [39558, 22693], [39557, 22693...",10,1,269182,"[208, 157, 250, 9, 158, 231, 10, 77, 127, 0, 89]",105.0,40.970562,0.786059,...,104.293274,0.607096,0.967742,0.656250,11.562446,8,35.355339,76.607521,59.920910,0.279197


In [23]:
pkls.loc[pkls['type']==12]=10 #replace cells in white to ecm since its just gap
pkls.loc[pkls['type']==1]=2 # all corneum cells are spinosum
# pkls.loc[pkls['type']==3]=4 # all shaft cells are follicle

In [24]:
# pkl = pkls[pkls['imID']==8]
# pkl = pkl.drop(columns=['bbox', 'centroid','contour'])
# pk = pkl[pkl['inroi']==1]
# p = pk[pk['type']==1]
# pkmean = p.mean(axis=0)
# pkmean['Ncell'] = len(p)
# pkmean['imID'] = 8
# pkmean['Age'] = agelut['Age'][agelut['redcap']==8].values[0]
# pkmean

In [25]:
%%time
avgs = []
stds = []
for imid in pkls['imID'].unique():
    pkl = pkls[pkls['imID']==imid]
    pkl = pkl.drop(columns=['bbox', 'centroid','contour'])
    for secid in pkl['inroi'].unique():
        pk = pkl[pkl['inroi']==secid]
        for type in range(2,12):
            p = pk[pk['type']==type]
            pkmean = p.mean(axis=0)

            pkmean['Ncell'] = len(p)
            pkmean['imID'] = imid
            pkmean['type'] = type
            pkmean['inroi'] = secid
            pkmean['Age'] = agelut['Age'][agelut['redcap']==imid].values[0]
            pkstd = p.std(axis=0)

            pkstd['imID'] = imid
            pkstd['type'] = type
            pkstd['inroi'] = secid
            pkstd['Age'] = agelut['Age'][agelut['redcap']==imid].values[0]
            stds.append(pkstd.replace(np.nan,0))
            avgs.append(pkmean.replace(np.nan, 0))

<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame re

CPU times: total: 1min 10s
Wall time: 1min 6s


<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
<timed exec>:10: FutureWarning: Dropping of nuisance columns in DataFrame re

In [31]:
avgdf = pd.DataFrame(avgs)
stddf = pd.DataFrame(stds)

In [27]:
avgdf.to_csv('avgdf.csv')
stddf.to_csv('stddf.csv')

In [39]:
avgs[134]

type              6.0
inroi             3.0
resident_area     0.0
c2t_distance      0.0
Area              0.0
Perimeter         0.0
Circularity       0.0
MA                0.0
ma                0.0
orientation       0.0
AspectRatio       0.0
Sol               0.0
Extent            0.0
EquiDia           0.0
imID             16.0
dist2nearest      0.0
oriA              0.0
oriB              0.0
local_align       0.0
Ncell             0.0
Age              15.5
dtype: float64

In [28]:
# avg_lvl_corr=[]
# for type in avgdf['type'].unique():
#     adf = avgdf[avgdf['type']==type]
#     corr = adf.corrwith(adf['age'], method='kendall')
#     avg_lvl_corr.append(corr)

In [43]:
df2 = pd.read_pickle(r"\\fatherserverdw\Q\research\images\skin_aging\wsi\hovernet_out\df\8.pkl")

In [44]:
df2

,bbox,centroid,contour,type,inroi,resident_area,c2t_distance,Area,Perimeter,Circularity,...,orientation,AspectRatio,Sol,Extent,EquiDia,imID,dist2nearest,oriA,oriB,local_align
0,"[[1627, 12118], [1646, 12127]]","[1881.754716981132, 11877.27358490566]","[[1886, 11867], [1885, 11868], [1884, 11868], ...",2,0,11382841,"[8190.0, 8190.0, 8190.0, 8190.0, 8190.0, 8190....",84.0,46.970562,0.478451,...,15.408552,0.481259,0.796209,0.491228,10.341766,8,88.769364,9.045911,33.463440,0.656105
1,"[[3740, 8056], [3749, 8071]]","[3966.7162162162163, 7840.310810810811]","[[3970, 7836], [3969, 7837], [3968, 7837], [39...",2,0,11382841,"[8190.0, 8190.0, 8190.0, 8190.0, 8190.0, 8190....",56.5,37.556349,0.503375,...,68.177139,0.364545,0.824818,0.418519,8.481629,8,12.041595,10.978894,60.039799,0.666896
2,"[[3747, 8029], [3768, 8047]]","[3941.6615384615384, 7852.626923076923]","[[3940, 7843], [3939, 7844], [3938, 7844], [39...",2,0,11382841,"[8190.0, 8190.0, 8190.0, 8190.0, 8190.0, 8190....",231.5,63.698484,0.716973,...,157.260712,0.770438,0.920477,0.612434,17.168429,8,13.038405,32.288494,60.039799,0.788801
3,"[[3748, 8045], [3760, 8058]]","[3954.541666666667, 7849.21875]","[[3954, 7844], [3953, 7845], [3952, 7845], [39...",2,0,11382841,"[8190.0, 8190.0, 8190.0, 8190.0, 8190.0, 8190....",79.5,37.213203,0.721411,...,60.039799,0.662782,0.893258,0.509615,10.060941,8,13.341664,10.978894,157.260712,0.978413
4,"[[3751, 8061], [3763, 8068]]","[3967.890625, 7852.359375]","[[3969, 7847], [3968, 7848], [3966, 7848], [39...",2,0,11382841,"[8190.0, 8190.0, 8190.0, 8190.0, 8190.0, 8190....",49.5,29.899495,0.695805,...,10.978894,0.620038,0.916667,0.589286,7.938851,8,12.041595,68.177139,60.039799,0.666896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83811,"[[47028, 18229], [47040, 18238]]","[46905.51851851852, 18361.962962962964]","[[46906, 18356], [46905, 18357], [46904, 18357...",10,1,269182,"[3154.0, 3074.0, 529.5, 1169.0, 575.0, 1791.0,...",65.5,32.727922,0.768448,...,170.316132,0.846964,0.935714,0.606481,9.132206,8,12.000000,22.067678,42.320881,1.027484
83812,"[[47040, 18229], [47053, 18239]]","[46905.030303030304, 18373.555555555555]","[[46903, 18368], [46903, 18369], [46902, 18370...",9,1,93829,"[3156.0, 3076.0, 534.5, 1164.0, 570.0, 1793.0,...",81.5,37.556349,0.726107,...,22.067678,0.616971,0.936782,0.626923,10.186708,8,12.000000,170.316132,42.320881,1.027484
83813,"[[47097, 18361], [47116, 18373]]","[47038.33783783784, 18433.391891891893]","[[47042, 18425], [47041, 18426], [47039, 18426...",9,1,93829,"[3224.0, 3146.0, 564.0, 1172.0, 588.0, 1862.0,...",126.5,49.698484,0.643597,...,20.871637,0.529522,0.923358,0.554825,12.691131,8,80.006250,128.926559,22.067678,1.082993
83814,"[[47124, 18284], [47138, 18300]]","[46962.931818181816, 18458.234848484848]","[[46956, 18452], [46956, 18456], [46957, 18457...",10,1,269182,"[3196.0, 3118.0, 564.0, 1139.0, 551.5, 1834.0,...",111.5,46.041630,0.660972,...,128.926559,0.459789,0.921488,0.497768,11.914957,8,80.006250,20.871637,22.067678,1.082993


In [45]:
np.max(df2.c2t_distance)

[8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0,
 8190.0]